encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
quantize_lut = True

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"

auto_train = False # 是否根据已运行的训练性能结果自动训练，（train_sam_num取已训练的最大值）

nbits_trained = 8
nbits_goal = 8
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train:
    ncodebooks = 128 # max:512
    ncentroids = 16
    train_sam_num = 100 # 训练集样本数
else:
    cb_ct_ntr_combinations_unique = change_nbits_auto_run_list(linear_name, method, feedback_bits, nbits_trained, nbits_goal)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])

# host_name = socket.gethostname()
# if host_name == 'DESKTOP-PLRL7TK':
#     dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
#     dir_result = ''
# elif host_name == 'DESKTOP-6FOH47P':
#     dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
#     dir_result = 'F:\\Projects\\python\\PQ\\res'
#     linearin_path_train= ''
#     linearout_path_train= ''
#     linearin_path_test = ''
#     linearout_path_test = ''
# elif host_name == 'jm-System-Product-Name':
#     dir_joined = '/data/hdr/transformer_data/joined'
#     dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
#     dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
#     dir_result = '/data/hdr/pq/res'
#     linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
#     linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
# else:
#     raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


# weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
# biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
# dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
# try:
#     os.mkdir(dir_result)
# except FileNotFoundError:
#     os.makedirs(dir_result)
# except FileExistsError:
#     pass 


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        bias_path=AMM_train_dirs["biaspath"])
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut)

running method:  Mithral
X.shape:  (102400, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    48.87861592479377
learn_multisplits(): returning loss:  9.690204874532427
learn_multisplits(): initial loss:    9724.825918970564
learn_multisplits(): returning loss:  131.4862334728241
learn_multisplits(): initial loss:    344.20393908444737
learn_multisplits(): returning loss:  36.511000621637265
learn_multisplits(): initial loss:    811.1675905607776
learn_multisplits(): returning loss:  117.7429518699646
learn_multisplits(): initial loss:    109.07573994303644
learn_multisplits(): returning loss:  6.04623505831367
learn_multisplits(): initial loss:    6783.18478452502
learn_multisplits(): returning loss:  36.99919390678406
learn_multisplits(): initial loss:    10895.418135684531
learn_multisplits(): returning loss:  144.59437704086304
learn_multisplits(): initial loss:    925.6868399221742
learn_multisplits(): returning loss:  208.0370578765869
learn_mul

In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PQ or method == METHOD_PLUTO or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_ql%i_nbits%i_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, quantize_lut, nbits, train_sam_num, test_sam_num, 
                                                        feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[ 0.46771798  0.21552014  0.3277554  ... -0.05278443  0.00682067
   0.11001119]
 [ 0.46771798  0.09027553  0.3277554  ...  0.07246019  0.00682067
   0.11001119]
 [ 0.5929626   0.09027553  0.453      ...  0.07246019  0.1320653
   0.11001119]
 ...
 [ 0.21722874  0.09027553  0.20251077 ...  0.1977048   0.2573099
   0.23525581]
 [ 0.34247336  0.09027553  0.3277554  ...  0.1977048   0.00682067
   0.48574507]
 [ 0.46771798  0.09027553  0.07726616 ...  0.44819403  0.00682067
   0.23525581]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
